In [6]:
from kafka import KafkaConsumer
import psycopg2
import json
import datetime

consumer = KafkaConsumer(
    'error_log',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='invalid-pg',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# Połączenie z PostgreSQL
conn = psycopg2.connect(
    dbname='mydb',
    user='myuser',
    password='myuserpass',
    host='postgres',
    port=5432
)
cursor = conn.cursor()

print("Kafka-to-Postgres processor running...")

for msg in consumer:
    data = msg.value
    event_time = datetime.datetime.fromtimestamp(data['event_timestamp'])
    cursor.execute(
        "INSERT INTO error_log (device_id, error_message, event_timestamp) VALUES (%s, %s, %s)",
        (data['device_id'], data['error_message'], event_time)
    )
    conn.commit()
    print(f"Inserted into DB: {data}")


Kafka-to-Postgres processor running...
Inserted into DB: {'device_id': 'sensor-002', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018109.914909}
Inserted into DB: {'device_id': 'sensor-002', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018112.9174767}
Inserted into DB: {'device_id': 'sensor-001', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018115.4203558}
Inserted into DB: {'device_id': 'sensor-002', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018117.4229424}
Inserted into DB: {'device_id': 'sensor-000', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018117.923822}
Inserted into DB: {'device_id': 'sensor-002', 'error_message': 'temperature or humidity not in accepted range', 'event_timestamp': 1748018118.9247904}
Inserted into DB: {'device_id': 'sensor-000', 'error_mess

KeyboardInterrupt: 